In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
n = 100

n_ = n // 2

a = np.concatenate((np.full(n_, 0), np.full(n_, 1)))

b = np.random.random_sample(size=a.size)

b_0 = b[a == 0]

b_1 = b[a == 1]

In [ ]:
b_grid = kraft.make_vector_grid(b, n_grid=8)

In [ ]:
b_keyword_arguments = dict(dimension_grids=(b_grid,), plot=False,)

b_grid, b_pdf = kraft.compute_element_x_dimension_joint_probability(
    b.reshape((b.size, 1)), **b_keyword_arguments,
)

b_grid = b_grid.reshape(b_grid.size)

b_0_pdf = kraft.compute_element_x_dimension_joint_probability(
    b_0.reshape((b_0.size, 1)), **b_keyword_arguments,
)[1]

b_1_pdf = kraft.compute_element_x_dimension_joint_probability(
    b_1.reshape((b_1.size, 1)), **b_keyword_arguments,
)[1]

a_b_js = (
    b_0.size * kraft.compute_kullback_leibler_divergence(b_0_pdf, b_pdf)
    + b_1.size * kraft.compute_kullback_leibler_divergence(b_1_pdf, b_pdf)
) / b.size

a_b_js_mi = a_b_js.sum()

print("(JS) MI = {}".format(a_b_js_mi))

print("(JS) IC = {}".format(np.sqrt(1 - np.exp(-2 * a_b_js_mi))))

In [ ]:
kraft.compute_information_coefficient_between_2_vectors(a, b)

In [ ]:
(a_grid, _b_grid), a_b_jpdf = kraft.unmesh(
    *kraft.compute_element_x_dimension_joint_probability(
        np.asarray((a, b)).T,
        dimension_grid_mins=(a.min(), b.min(),),
        dimension_grid_maxs=(a.max(), b.max(),),
        dimension_fraction_grid_extensions=(fraction_grid_extension,) * 2,
        dimension_n_grids=(n_grid,) * 2,
        plot=False,
    )
)

a_grid_d = a_grid[1] - a_grid[0]

print(a_grid_d)

b_grid_d = _b_grid[1] - _b_grid[0]

print(b_grid_d)

a_mpdf = a_b_jpdf.sum(axis=1) * b_grid_d

b_mpdf = a_b_jpdf.sum(axis=0) * a_grid_d

a_b_mpdf = np.outer(a_mpdf, b_mpdf)

mi = (
    kraft.compute_kullback_leibler_divergence(a_b_jpdf, a_b_mpdf).sum()
    * a_grid_d
    * b_grid_d
)

print("MI = {}".format(mi))

print("IC = {}".format(np.sqrt(1 - np.exp(-2 * mi))))

In [ ]:
n = 10

a = np.arange(1, n) / n

b = a[::-1]

c = (a + b) / 2

jsa = a * np.log(a / c)

jsb = b * np.log(b / c)

js = jsa + jsb

kraft.plot_plotly_figure(
    {
        "data": [
            {"name": "A", "y": a},
            {"name": "B", "y": b},
            {"name": "Center", "y": c},
            {"name": "JS A", "y": jsa,},
            {"name": "JS B", "y": jsb},
            {"name": "JS", "y": js},
        ]
    }
)

In [ ]:
a = np.asarray([0, 0, 0, 1, 1, 1])

b = np.arange(a.size)

p_0 = (a == 0).sum() / a.size

p_1 = (a == 1).sum() / a.size

b_0 = b[a == 0]

b_1 = b[a == 1]

p_b = kraft.estimate_element_x_dimension_kernel_density(
    b.reshape((b.size, 1)),
    dimension_grid_mins=(b.min(),),
    dimension_grid_maxs=(b.max(),),
    dimension_fraction_grid_extensions=(1e-3,),
    dimension_n_grids=(8,),
)[1]

p_b_0 = kraft.estimate_element_x_dimension_kernel_density(
    b_0.reshape((b_0.size, 1)),
    dimension_grid_mins=(b.min(),),
    dimension_grid_maxs=(b.max(),),
    dimension_fraction_grid_extensions=(1e-3,),
    dimension_n_grids=(8,),
)[1]

p_b_1 = kraft.estimate_element_x_dimension_kernel_density(
    b_1.reshape((b_1.size, 1)),
    dimension_grid_mins=(b.min(),),
    dimension_grid_maxs=(b.max(),),
    dimension_fraction_grid_extensions=(1e-3,),
    dimension_n_grids=(8,),
)[1]

js = p_0 * kraft.compute_kullback_leibler_divergence(
    p_b_0, p_b
) + p_1 * kraft.compute_kullback_leibler_divergence(p_b_1, p_b)

print(js.sum())

In [ ]:
grid, pab = kraft.compute_element_x_dimension_joint_probability(
    np.asarray((a, b)).T,
    dimension_grid_mins=(a.min(), b.min()),
    dimension_grid_maxs=(a.max(), b.max()),
    dimension_fraction_grid_extensions=(1e-3, 1e-3),
    dimension_n_grids=(8, 8),
)

p_ab = p_ab.reshape((8, 8))

p_a_p_b = np.outer(p_a, p_b)

In [ ]:
grid[:, 0]
grid[:, 1]

In [ ]:
pxpy = asarray((px,) * n_grid).T * asarray((py,) * n_grid)

In [ ]:
(p_ab * np.log(p_ab / p_a_p_b)).sum().sum()